In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import healpy as hp, tensorflow as tf
from quick_infer import load_dsphere, run_full, run_encoder, run_decoder
import numpy as np

from ae_training import load_data_from_folders

model = load_dsphere("training_output/outer_final_evaluated_model.keras",
                     gpu=False)           # now loads without error


leaflet_type = 'outer'
params = np.load("training_output/outer_radial_normalization_params.npz")
radial_mean = params['mean']
radial_std = params['std']
folders = [
    d for d in os.listdir(".")
    if d.startswith("configf_") and os.path.isdir(d)
]
print(len(folders))
print(folders[0:5])


2025-06-02 13:06:36.716353: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748862396.733310  421101 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748862396.739080  421101 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748862396.754647  421101 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748862396.754667  421101 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748862396.754668  421101 computation_placer.cc:177] computation placer alr

25-06-02 13:06:40  sbi_flows INF   WARNING: This network assumes that everything concerning healpy is in NEST ordering... 
25-06-02 13:06:40  sbi_flows INF   Detected a reduction factor of 1.0, the input with nside 16 will be transformed to 16 during a forward pass. Checking for consistency with indices... 
25-06-02 13:06:40  sbi_flows INF   indices seem consistent... 
25-06-02 13:06:41  sbi_flows INF   WARNING: This network assumes that everything concerning healpy is in NEST ordering... 
25-06-02 13:06:41  sbi_flows INF   Detected a reduction factor of 1.0, the input with nside 8 will be transformed to 8 during a forward pass. Checking for consistency with indices... 
25-06-02 13:06:41  sbi_flows INF   indices seem consistent... 
25-06-02 13:06:41  sbi_flows INF   WARNING: This network assumes that everything concerning healpy is in NEST ordering... 
25-06-02 13:06:41  sbi_flows INF   Detected a reduction factor of 1.0, the input with nside 4 will be transformed to 4 during a forward

2025-06-02 13:06:41.454894: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


25-06-02 13:06:41  sbi_flows INF   WARNING: This network assumes that everything concerning healpy is in NEST ordering... 
25-06-02 13:06:41  sbi_flows INF   Detected a reduction factor of 1.0, the input with nside 8 will be transformed to 8 during a forward pass. Checking for consistency with indices... 
25-06-02 13:06:41  sbi_flows INF   indices seem consistent... 
25-06-02 13:06:41  sbi_flows INF   WARNING: This network assumes that everything concerning healpy is in NEST ordering... 
25-06-02 13:06:41  sbi_flows INF   Detected a reduction factor of 1.0, the input with nside 16 will be transformed to 16 during a forward pass. Checking for consistency with indices... 
25-06-02 13:06:41  sbi_flows INF   indices seem consistent... 
25-06-02 13:06:41  sbi_flows INF   WARNING: This network assumes that everything concerning healpy is in NEST ordering... 
25-06-02 13:06:41  sbi_flows INF   Detected a reduction factor of 1.0, the input with nside 16 will be transformed to 16 during a forwa

In [ ]:
for idx, folder in enumerate(folders, start=1):
    if idx % 50 == 0:
        print(f"Processing folder #{idx}: {folder}")
    ready = load_data_from_folders([folder], leaflet_type=leaflet_type, initial_nside=16)
    ready[..., 0] = (ready[..., 0] - radial_mean) / radial_std
    n_frames = ready.shape[0]              # e.g. 90, 101, 120, etc.
    # 3) allocate latent with the correct first dimension
    latent = np.zeros((n_frames, 48, 32), dtype=float)
    #print(latent.shape)   # → (101, 32, 48)
    for i in range(n_frames):
        frame = ready[i:i+1] 
        frame_enc = model.encode(frame)
        #print('encoded shape', frame_enc.shape, 'original shape', frame.shape, 'latent shape', latent[i:i+1].shape)
        latent[i:i+1] = frame_enc
        #frame_dec = model.decode(frame_enc)
        #for element in frame_dec:
        #    print(frame_dec[element].shape)
    np.save(f"{folder}/{leaflet_type}_ae_latent", latent)
    


before r2n (101, 3072, 4)
after r2n: (101, 3072, 4)
before r2n (101, 3072, 4)
after r2n: (101, 3072, 4)
before r2n (101, 3072, 4)
after r2n: (101, 3072, 4)
before r2n (101, 3072, 4)
after r2n: (101, 3072, 4)
before r2n (101, 3072, 4)
after r2n: (101, 3072, 4)
before r2n (101, 3072, 4)
after r2n: (101, 3072, 4)
before r2n (101, 3072, 4)
after r2n: (101, 3072, 4)
before r2n (101, 3072, 4)
after r2n: (101, 3072, 4)
before r2n (101, 3072, 4)
after r2n: (101, 3072, 4)
before r2n (101, 3072, 4)
after r2n: (101, 3072, 4)
before r2n (101, 3072, 4)
after r2n: (101, 3072, 4)


In [3]:
import os
import numpy as np

leaflet_type = 'outer'
radial_filename = "area_outer_radial_dist.npy"

# Step 2: Check each folder's radial array shape
bad_folders = []

for folder in folders:
    radial_path = os.path.join(folder, "analysis_output", radial_filename)
    if not os.path.isfile(radial_path):
        # If the file doesn't exist, count as bad with 0 frames
        bad_folders.append((folder, 0))
    else:
        arr = np.load(radial_path)
        n_frames = arr.shape[0]
        if n_frames != 101:
            bad_folders.append((folder, n_frames))

# Step 3: Report
num_bad = len(bad_folders)
print(f"Number of folders without exactly 101 frames: {num_bad}")
for folder, n in bad_folders:
    print(f"  {folder} has {n} frames")


Number of folders without exactly 101 frames: 43
  configf_1400 has 90 frames
  configf_10 has 68 frames
  configf_3370 has 89 frames
  configf_2500 has 59 frames
  configf_3214 has 55 frames
  configf_568 has 91 frames
  configf_3649 has 30 frames
  configf_126 has 83 frames
  configf_3269 has 80 frames
  configf_2112 has 95 frames
  configf_424 has 95 frames
  configf_767 has 77 frames
  configf_3199 has 86 frames
  configf_1735 has 95 frames
  configf_2332 has 83 frames
  configf_3364 has 95 frames
  configf_3850 has 77 frames
  configf_654 has 100 frames
  configf_263 has 80 frames
  configf_3619 has 96 frames
  configf_1836 has 86 frames
  configf_3256 has 79 frames
  configf_1051 has 85 frames
  configf_2894 has 79 frames
  configf_30 has 100 frames
  configf_430 has 70 frames
  configf_1252 has 86 frames
  configf_3580 has 83 frames
  configf_3070 has 98 frames
  configf_2230 has 84 frames
  configf_2322 has 86 frames
  configf_4086 has 94 frames
  configf_2966 has 75 frames
  c